In [50]:
# load libs
import numpy as np 
import scipy 
import pandas as pd
import os
import matplotlib.pyplot as plt #import matplotlib as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import seaborn as sns #import mat73
import pickle as pkl
from datetime import datetime
import mne
from mne.viz import plot_topomap
from mne.io import RawArray
import numpy.matlib
sns.set_theme() # set the plotting atmosphere

In [56]:
NumHarms = 4
prebins = 1
NumBins = 6 # 6 contrasts
allbins = prebins+NumBins # 6 contrasts + prelude bin (7)
NumChans = 128
NumHemis = 2 # hemifield data
NumFriters = NumHarms*NumHemis # 8 : 2f1 4f1 6f1 8f1 2f2 4f2 6f2 8f2
co = 0 # condition out of 6 
### row count to determine if 1f1 data exists
ExpectedRows = (8*allbins)* 2 # 112 
BadRows = (9*allbins)*2 # 126
h = np.arange(0,NumFriters,1)
# set freq index list for 2f1 - 8f2 data
hIters = np.sort(np.tile(h,allbins))
chanInd = np.arange(1,NumChans,1) # channel index ...

In [54]:
# montage info 
montageIn = mne.channels.make_standard_montage("GSN-HydroCel-128")
info = mne.create_info(ch_names=montageIn.ch_names, sfreq=1, ch_types="eeg")
info.set_montage(montageIn)
print(montageIn)
# contrast labels
# contrast_levels=np.array([1, 3, 5, 16, 40, 100])
# cll=['1%', '3%', '5%', '16%', '40%', '100%']
# ttl = ['2f1','4f1','6f1','8f1','2f2','4f2','6f2','8f2']
# occipital electrode placement locs
elec_Arc1_outer = [68,64,58,51,47,42,37,31,    80,87,93,98,97,96,95,94]
elec_Arc2 = [73,69,65,59,52,53,54,  55,  79,86,92,91,90,89,88]
elec_Arc3 = [74,70,66,60,61,  78,85,84,83,82,  81]
elec_Arc4_inner = [71,67,62,77,76,75,     72]
AllAnalysisChans = np.concatenate([elec_Arc1_outer,elec_Arc2,elec_Arc3,elec_Arc4_inner])
AnalysisChans = np.unique(AllAnalysisChans) - 1
VisChans = np.sort(AnalysisChans)
print(f'Channels to be visualized: {VisChans}')
ocm = np.zeros_like((np.arange(0,NumChans,1)))
ocm[VisChans] = True # set good chans to true
#ocm_chans = ocm.astype(bool)

# function to plot topos
def init_TopoTemplate(title_In):
    """Prep topo plot fig and axes"""
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(8,4))
    axes[0].axis('off')
    axes[1].axis('off')
    axes[0].gridspec_kw=dict(height_ratios=[3])
    axes[1].gridspec_kw=dict(height_ratios=[3])
    plt.tight_layout()
    plt.suptitle(f'{title_In}', fontsize = 20)
    return fig, axes

<DigMontage | 0 extras (headshape), 0 HPIs, 3 fiducials, 128 channels>
Channels to be visualized: [30 36 41 46 50 51 52 53 54 57 58 59 60 61 63 64 65 66 67 68 69 70 71 72
 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96
 97]


### Import all files that will be sorted and exported into another .mat file

In [53]:
# Main Directory of processed file from MatLab
MainDir = 'C:\\plimon\\LTP_analysis\\eegMatFiles\\MAT' # set dir
os.chdir(MainDir) # change old dir, to this dir
d = os.listdir(MainDir) # list files in dir
print(f'Number of Files on hand: {int(len(d))}')
Fnames = np.sort(d) # get all subjects label names 

Number of Files on hand: 50


In [74]:
SignalData  = {} # ave average data 
RealnImaginaryData = {} # save real and imaginary data 

for suIn in range(len(d)):#range(10):# og d 
    #RealnImaginaryData[0] = {}
### import file via path 
    FileN = Fnames[suIn+46] # import from sorted subj expt name list                       
    file_path = os.path.join(MainDir, FileN) # join paths and prep 2 load
    mat_data = scipy.io.loadmat(file_path) # import data file

### extract signal data from file and dims
    MainData = mat_data['signalData'][co][0]
    SortDataIn = np.array(MainData)
    [NumRows, NumChans, NumTrials] = SortDataIn.shape # get dims of data to use later ..
    real_imag_bound = int((NumRows/2)) # real and imaginary freq values ...
    prepost_bound = int(NumTrials/2)

### reshape real and imaginary values and remove 0's with nan's
    dIn = SortDataIn.reshape(real_imag_bound,2,NumChans,NumTrials,order='A')
    # RealnImaginaryData[suIn]= dIn    
    dIn[dIn==0]=np.nan
    #print(dIn.shape)
    RowCount = int(dIn.shape[0])
    if RowCount == int(ExpectedRows/2): # if data dont have extra f1 data
        print(f'data is expected shape {dIn.shape}')
        data = dIn
    else:
        #print(f'data contains f1 info') # data has extra f1 info - rmv
        data = dIn[allbins:,:,:] # remove 0-7 rows
        print(f'f2 data removed, new shape {data.shape}')
RealnImaginaryData[suIn] = data

#     xF_Bins = {}
#     for hi in range(8):
#         tempIn =data[hIters == hi]
#         tempOut = tempIn[1:,:,:,:]
#         #print(tempOut.shape)
#         xF_Bins[hi] = tempOut

#     tInds = np.arange(0,NumTrials,prepost_bound)# trial inds

#     sOut = {} # hypot data
#     # riOut = {} # real / imaginary data

#     valMat = np.zeros((8,2,6,128))
#     for fritter in range(8): # per frequency 
#         fIn = xF_Bins[fritter]
#         #print(fIn.shape)
#         for triters in range(2): # per trial
#             ti = tInds[triters]
#             te = ti + prepost_bound
#             ppIn = fIn[:,:,:,ti:te]
#             for biters in range(6): # per bin
#                 bdIn = ppIn[biters,:,:,:]
#                 for chiters in range(128): # per channel
#                     chIn = bdIn[:,chiters,:] # 2 x 29 (real /imag) x trials
#                     preChAct = np.nanmean(bdIn[0,chiters,:]) # 39 rela and imaginary vals per bin x chan x harmonic x hemifield
#                     postChAct = np.nanmean(bdIn[1,chiters,:])
#                     hypOut = np.hypot(preChAct,postChAct)
#                     valMat[fritter,triters,biters,chiters] = hypOut
#                     #print(preChAct,postChAct, hypOut)
# print(valMat.shape)

39
data is expected shape (56, 2, 128, 78)
39
data is expected shape (56, 2, 128, 78)
40
f2 data removed, new shape (56, 2, 128, 80)


In [ ]:
t_F1  = valMat[0,:,:,:] # harmonicHemi x prepost x bins x channels
t_F2 = valMat[4,:,:,:]
print(t_F1.shape) # pre post for contrat allchannels

In [ ]:
for bin in range(6):
    tIn_pre = np.array(t_F1[0,bin,:])
    tIn_post = np.array(t_F1[1,bin,:])
    # findLims = np.vstack((tIn_F1,tIn_F2))
    # print(findLims.shape)
    # t_max = np.max(findLims)
    # t_min = np.min(findLims)
    # print(t_min,t_max)
    v_max = 2
    v_min = 0

    fig, axes = init_TopoTemplate(f'pre / post 2f1 {cll[bin]} contrast')
    # left side plot
    im,_ = plot_topomap(tIn_pre, info,mask = ocm, vlim = [v_min,v_max],axes = axes[0], show =False,extrapolate = 'local',cmap = 'Spectral_r', res = 32, contours =4,sphere=(0.0,0.0,0.0,0.09),outlines = 'head')
    # middle  plot
    im,_ = plot_topomap(tIn_post, info,mask = ocm,vlim = [v_min,v_max], axes = axes[1],show =False,extrapolate = 'local',cmap = 'Spectral_r', res = 32, contours =4,sphere=(0.0,0.0,0.0,0.09),outlines = 'head')

    divider = make_axes_locatable(axes[1])
    cax = divider.append_axes('right', size='5%', pad=0.1)
    cbar = plt.colorbar(im, cax=cax)

In [ ]:
for bin in range(6):
    tIn_pre = t_F2[0,bin,:]
    tIn_post = t_F2[1,bin,:]
    # findLims = np.vstack((tIn_F1,tIn_F2))
    # print(findLims.shape)
    # t_max = np.max(findLims)
    # t_min = np.min(findLims)
    # print(t_min,t_max)
    v_max = 2
    v_min = 0

    fig, axes = init_TopoTemplate(f'pre / post 2f2 {cll[bin]} contrast')
    # left side plot
    im,_ = plot_topomap(tIn_pre, info,mask = ocm, vlim = [v_min,v_max],axes = axes[0], show =False,extrapolate = 'local',cmap = 'Spectral_r', res = 32, contours =4,sphere=(0.0,0.0,0.0,0.09),outlines = 'head')
    # middle  plot
    im,_ = plot_topomap(tIn_post, info,mask = ocm,vlim = [v_min,v_max], axes = axes[1],show =False,extrapolate = 'local',cmap = 'Spectral_r', res = 32, contours =4,sphere=(0.0,0.0,0.0,0.09),outlines = 'head')

    divider = make_axes_locatable(axes[1])
    cax = divider.append_axes('right', size='5%', pad=0.1)
    cbar = plt.colorbar(im, cax=cax)

## dont touch!!! until above is resolved

In [ ]:
print(RealnImaginaryData.keys())
print(RealnImaginaryData[0].keys())
print(RealnImaginaryData[0][0].shape)

In [ ]:
# sIn = 25

# fig,axs = plt.subplots(8,2, figsize = (4,16), sharey = True)
# d2p = SignalData[sIn]
# print(d2p.keys())

# for i in range(8):
#     #print(d2p[i].shape)
#     axs[i,0].plot(d2p[i][:,0,55:90])
#     axs[i,1].plot(d2p[i][:,1,55:90])


#### Sort Condition Bools

In [ ]:
FileNamesIndicies = np.array(d) # make list into array 
FileName = [FileNamesIndicies.replace('_','-') for FileNamesIndicies in FileNamesIndicies] # uniform fnames to make bools for attnl and attnR 

In [ ]:
DataOut = {}

DataOut['RealImaginaryData_sIns'] = RealnImaginaryData
DataOut['EEGData'] = SignalData
DataOut['SubIDs'] = FileName

In [ ]:
SaveDataDir = 'C:\\plimon\\LTP_analysis\\eegMatFiles\\AllMat'
print(os.path.exists(SaveDataDir)) # does pth exist :p

dnt = datetime.now() # add date and time bc im wreckless when saving ..
fdnt = dnt.strftime("%Y%m%d_%H%M%S") # set the above as a string ...
FileOutName = 'AllSubj_MatFiles_ri_C1'
FileN = f'{FileOutName}_{fdnt}.pkl' 
NewFileNPath = os.path.join(SaveDataDir,FileN)
print('Full New File Dir: ', NewFileNPath)

saveFile = 'y'
if saveFile == 'y':
 with open(NewFileNPath, 'wb') as file:
    pkl.dump(DataOut, file, protocol=pkl.HIGHEST_PROTOCOL)
    print('MAT Data For All Subjs Saved! :))')
else:
    print('Did Not Save File! Change file name before switching to y!')